## Using the BISON API
The USGS provides an API for accessing species observation data. https://bison.usgs.gov/doc/api.jsp

This API is much better documented than the NWIS API, and we'll use it to dig a bit deeper into how the `requests` package can faciliate data access via APIs. 

* We'll begin by replicating the example API call they show on their web page:<br> 
[https://bison.usgs.gov/api/search.json?species=Bison bison&type=scientific_name&start=0&count=1](
https://bison.usgs.gov/api/search.json?species=Bison%20bison&type=scientific_name&start=0&count=1)

In [ ]:
#First, import the wonderful requests module
import requests

* Now, we'll deconstruct the example URL into the service URL and parameters, saving the paramters as a dictionary. Note we are just providing a few of the parameters available through the [API](https://bison.usgs.gov/doc/api.jsp#opensearch). We could add more search criteria if we wanted, but for now we just want to grab the first 500 Bison records. 

In [ ]:
# Construct the service URL as two components: the service URL and the request parameters
url = 'http://bison.usgs.gov/api/search.json'
params = {'species':'Bison bison',
          'type':'scientific_name',
          'start':'0',
          'count':'500'
         }

* With the components set as variables, we use the `requests.get()` function to send our request off to the server at the address provided, storing the servers response as a variable called `response`. 

In [ ]:
#Send the request to the server and store the response as a variable
response = requests.get(url,params)

* This response object contains a number of properties and methods. Let's have a look at the reponse in raw text format. 

In [ ]:
#View the reponse in text format
response.text

**Yikes**, that's much less readable than the NWIS output!

Well, that's because the response from the BISON server is in **JSON** format. JSON, short for *JavaScript Object Notation*, is a text document that stores information in `key`:`value` pairs, *much like a Python dictionary*. Still, it's a raw text object, but one that we convert into a Python dictionary using Python's json package.

In [ ]:
#Import the module
import json

#Convert the response 
data = json.loads(response.text)
type(data)

> *Note*: we could also convert this to JSON using the `json` function of the `response` object...<br>The code below has the exact same results as the one above. 

In [ ]:
data=response.json()
type(data)

* Ok, if it's a dictionary, what are it's keys? 

In [ ]:
#List the keys in the returned JSON object
data.keys()

* What are the values linked with the 'data' key?

In [ ]:
#Show the value associated with the `data` key
data['data']

* Oh, it's a list of occurrences! Let's examine the first one...

In [ ]:
#Display the first "data" value
data['data'][0]

* We see it's a dictionary too! Let's list the `decimalLatitude` item value...

In [ ]:
#We can get the latitude of the record from it's `decimalLatitude` key
data['data'][0]['decimalLatitude']

► **So** we see the Bison observations are stored as list of dictionaries which are accessed within the `data` key in the results dictionary generated from the JSON response to our API request. (Phew!)

* With a bit more code we can loop through all the data records and print out the lat and long coordinates...

In [ ]:
#Loop thorough each observation and print the lat and long values
for observation in data['data']:
    print (observation['decimalLatitude'],observation['decimalLongitude'])

► *If the above throws an error, can you debug it? HINT: the `geo` tag indicates whether coordinate info exist for the record...*

### [Another] Preview of 'Pandas' - that clever Python package with many uses!
Pandas can create a "data frame" from dictionary values. We'll talk about this soon, but can be quite useful!

In [ ]:
import pandas as pd
df = pd.DataFrame(data['data'])
df.head()

And Pandas allows us to do some nifty analyses, including subsetting records for a specific provider.
* First we'll get a list of unique providers found in the data

In [ ]:
#Generate a list of providers
df.provider.unique()

* Now, we'll subset the rows that include that provider...

In [ ]:
df.query("provider == 'Denver Museum of Nature & Science'")

## Exercise:
* Extract the first 500 red wolf (*"Canis rufus"*) records from the BISON API. 
* Can you create a table listing the records collected by the `University of Kansas Biodiversity Institute`?
* *Challenge*: Can you create a table listing all the records collected in North Carolina?

In [57]:
#importing requests
import requests

# Construct the service URL as two components: the service URL and the request parameters
url = 'http://bison.usgs.gov/api/search.json'
params = {'species':'Canis rufus',
          'type':'scientific_name',
          'start':'0',
          'count':'1000',
         }

#Send the request to the server and store the response as a variable
response = requests.get(url,params)

#response.text

In [58]:
#Importing json
import json

#Convert the response 
data = json.loads(response.text)

In [59]:
import pandas as pd
df = pd.DataFrame(data['data'])
df.head()

,basis,common_name,decimalLatitude,decimalLongitude,geo,name,occurrenceID,provider
0,Specimen,,NaN,NaN,No,Canis rufus,82909859,"National Museum of Natural History, Smithsonia..."
1,Specimen,,NaN,NaN,No,Canis rufus,82910676,"National Museum of Natural History, Smithsonia..."
2,Specimen,,NaN,NaN,No,Canis rufus,82911142,"National Museum of Natural History, Smithsonia..."
3,Specimen,,NaN,NaN,No,Canis rufus,82911261,"National Museum of Natural History, Smithsonia..."
4,Specimen,,NaN,NaN,No,Canis rufus,82912752,"National Museum of Natural History, Smithsonia..."


In [60]:
df.query("provider == 'University of Kansas Biodiversity Institute'")

,basis,common_name,decimalLatitude,decimalLongitude,geo,name,occurrenceID,provider
526,Specimen,,29.729999542236328,-96.31999969482422,Yes,Canis rufus,185722766,University of Kansas Biodiversity Institute
546,Specimen,,29.579999923706055,-96.51000213623047,Yes,Canis rufus,185220474,University of Kansas Biodiversity Institute
548,Specimen,,29.530000686645508,-96.56999969482422,Yes,Canis rufus,185005282,University of Kansas Biodiversity Institute
549,Specimen,,29.530000686645508,-96.56999969482422,Yes,Canis rufus,185005284,University of Kansas Biodiversity Institute
550,Specimen,,29.530000686645508,-96.56999969482422,Yes,Canis rufus,185005286,University of Kansas Biodiversity Institute
551,Specimen,,27.809999465942383,-97.06999969482422,Yes,Canis rufus,182113165,University of Kansas Biodiversity Institute
553,Specimen,,29.59000015258789,-96.63999938964844,Yes,Canis rufus,184574899,University of Kansas Biodiversity Institute
557,Specimen,,29.6299991607666,-96.62000274658203,Yes,Canis rufus,184686921,University of Kansas Biodiversity Institute
591,Specimen,,29.239999771118164,-95.16999816894531,Yes,Canis rufus,190415693,University of Kansas Biodiversity Institute
592,Specimen,,29.239999771118164,-95.16999816894531,Yes,Canis rufus,190415694,University of Kansas Biodiversity Institute


In [79]:
##Trying to get just the North Carolina ones. CHALLENGE QUESTION

url2 = 'http://bison.usgs.gov/api/search.json'
params2 = {'species':'Canis rufus',
          'type':'scientific_name',
           'state' : "North Carolina"
           #"data":[{'states':"North Carolina"}]  These don't work...
          #'states':{"North Carolina"}
         }

#Send the request to the server and store the response as a variable
response2 = requests.get(url2,params2)

data2 = json.loads(response2.text)

df2 = pd.DataFrame(data2['data'])
df2.head()

,basis,common_name,decimalLatitude,decimalLongitude,geo,name,occurrenceID,provider
0,Specimen,,35.5756721496582,-76.29776000976562,Yes,Canis rufus,303013045,James R. Slater Museum of Natural History
1,Specimen,,35.83486557006836,-75.91395568847656,Yes,Canis rufus,306819508,James R. Slater Museum of Natural History
2,Specimen,,35.83486557006836,-75.91395568847656,Yes,Canis rufus,306819509,James R. Slater Museum of Natural History
3,None,,35.11817932128906,-77.0825424194336,Yes,Canis rufus,2108627,BISON
4,None,,35.4823112487793,-76.8420181274414,Yes,Canis rufus,2108762,BISON
